In [13]:
import tweepy
import pandas as pd
import authentification
import datetime


In [2]:
screen_name = "1O17SQD"
TWEET_MODE = 'extended'
api = authentification.credentials()

def get_tweets(screen_name, api):
    """
    Retrieves tweets from the specified user using the specified Twitter API.

    Parameters
    ----------
    screen_name : str
        Screen name of the user whose tweets are to be retrieved.
    api: tweepy.API
        Twitter API object used to access the user's tweets.

    Returns
    -------
    list of tweepy.models.Status
        List of the user's tweets.
    """
    tweets = api.user_timeline(screen_name=screen_name,
                               # 200 is the maximum number allowed
                               count=200,
                               exclude_replies=True,
                               include_rts=False,
                               # Necessary to keep the full text
                               # otherwise only the first 140 words are extracted
                               tweet_mode=TWEET_MODE
                               )
    return tweets

def get_all_tweets(screen_name, api):
    """
    Retrieves all tweets from the specified user using the specified Twitter API.

    Parameters
    ----------
    screen_name : str
        Screen name of the user whose tweets are to be retrieved.
    api: tweepy.API
        Twitter API object used to access the user's tweets.

    Returns
    -------
    list of tweepy.models.Status
        List of all the user's tweets.
    """
    tweets = get_tweets(screen_name, api)
    all_tweets = []
    all_tweets.extend(tweets)
    oldest_id = tweets[-1].id
    while True:
        tweets = api.user_timeline(screen_name=screen_name,
                                   # 200 is the maximum number allowed
                                   count=200,
                                   exclude_replies=True,
                                   include_rts=False,
                                   max_id=oldest_id - 1,
                                   # Necessary to keep the full text
                                   # otherwise only the first 140 words are extracted
                                   tweet_mode=TWEET_MODE
                                   )
        if not len(tweets):
            break
        oldest_id = tweets[-1].id
        all_tweets.extend(tweets)

    return all_tweets

In [17]:
def tweet_source(screen_name, api):
    tweet_source = []
    tweet_created_year = []
    for status in get_all_tweets(screen_name, api):
        tweet_source.append(status.source)
        tweet_created_year.append(status.created_at.strftime("%Y"))
    df = {'tweet_source': tweet_source, 'tweet_created_year': tweet_created_year}
    df = pd.DataFrame(df)
    return df
df = tweet_source(screen_name, api)

In [22]:
df['tweet_created_year'].unique()

array(['2023', '2022', '2021', '2020', '2019', '2018', '2017'],
      dtype=object)

In [33]:
df.groupby(['tweet_source', 'tweet_created_year'])['tweet_source'].count().reset_index(name='count')

,tweet_source,tweet_created_year,count
0,Twitter Web App,2020,119
1,Twitter Web App,2021,15
2,Twitter Web App,2022,188
3,Twitter Web App,2023,9
4,Twitter Web Client,2017,124
5,Twitter Web Client,2018,13
6,Twitter Web Client,2020,1
7,Twitter for Android,2017,1
8,Twitter for Android,2018,2
9,Twitter for Android,2020,1
